In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd 'drive/My Drive'

/content/drive/My Drive


In [ ]:
import pandas as pd
df = pd.read_csv("wiki_movie_plots_deduped.csv")
df.head(3)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."


In [ ]:
df.shape

(34886, 8)

In [ ]:
import numpy as np
np.unique(df['Origin/Ethnicity'])

array(['American', 'Assamese', 'Australian', 'Bangladeshi', 'Bengali',
       'Bollywood', 'British', 'Canadian', 'Chinese', 'Egyptian',
       'Filipino', 'Hong Kong', 'Japanese', 'Kannada', 'Malayalam',
       'Malaysian', 'Maldivian', 'Marathi', 'Punjabi', 'Russian',
       'South_Korean', 'Tamil', 'Telugu', 'Turkish'], dtype=object)

In [ ]:
df2 = df.loc[(df['Origin/Ethnicity']=='American') & (df['Release Year']>2000)]
len(df2)

3595

In [ ]:
my_df = pd.DataFrame(df2)

In [ ]:
my_df.tail(3)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
17374,2017,All the Money in the World,American,Ridley Scott,Ridley Scott (director); David Scarpa (screenp...,"biography, crime, drama",https://en.wikipedia.org/wiki/All_the_Money_in...,"In 1973, 16-year-old John Paul Getty III (Paul..."
17375,2017,Molly's Game,American,Aaron Sorkin,Aaron Sorkin (director/screenplay); Jessica Ch...,"biography, crime, drama",https://en.wikipedia.org/wiki/Molly%27s_Game,Molly Bloom is world-class mogul skier with Ol...
17376,2017,Phantom Thread,American,Paul Thomas Anderson,Paul Thomas Anderson (director/screenplay); Da...,drama,https://en.wikipedia.org/wiki/Phantom_Thread,"In 1954 London, renowned fashion designer Reyn..."


In [ ]:
 final_df = my_df[["Title","Plot"]]
 final_df = final_df.set_index('Title')
 final_df.head(3)

,Plot
Title,
15 Minutes,Ex-convicts Emil Slovak (Karel Roden) and Oleg...
3000 Miles to Graceland,"Outside of Las Vegas, Michael Zane, a recent p..."
A.I. Artificial Intelligence,"In the late 22nd century, rising sea levels fr..."


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('average_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Error loading average_perceptron_tagger: Package
[nltk_data]     'average_perceptron_tagger' not found in index
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
def preprocess_sentences(text):
  text = text.lower()
  words = nltk.word_tokenize(text)
  my_sent=[lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]
  finalSent = ''.join(my_sent)

  finalSent = finalSent.replace("n't"," not")
  finalSent = finalSent.replace("'m"," am")
  finalSent = finalSent.replace("'s"," is")
  finalSent = finalSent.replace("'re"," are")
  finalSent = finalSent.replace("'ll"," will")
  finalSent = finalSent.replace("'ve"," have")
  finalSent = finalSent.replace("'d"," would")
  return finalSent

final_df["new_plot"] = final_df["Plot"].apply(preprocess_sentences)
final_df.head()

,Plot,new_plot
Title,,
15 Minutes,Ex-convicts Emil Slovak (Karel Roden) and Oleg...,ex-convictsemilslovak(karelroden)olegrazgul(ol...
3000 Miles to Graceland,"Outside of Las Vegas, Michael Zane, a recent p...","outsidelavega,michaelzane,recentparolee,stopru..."
A.I. Artificial Intelligence,"In the late 22nd century, rising sea levels fr...","late22ndcentury,risingsealevelglobalwarmingwip..."
The Affair of the Necklace,"Jeanne de Saint-Rémy de Valois, orphaned at an...","jeannedesaint-rémydevalois,orphanedearlyage,de..."
Ali,"The film begins with Cassius Clay, Jr. before ...","filmbegincassiusclay,jr.championshipdebutheavy..."


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_movieid = tfidf.fit_transform((final_df["new_plot"]))

#Find Cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(tfidf_movieid,tfidf_movieid)


In [ ]:
indices = pd.Series(final_df.index)

def recommendation(title,cosine_sim = similarity):
  try:
    index = indices[indices == title].index[0]
    similarity_scores = pd.Series(cosine_sim[index]).sort_values(ascending=False)
    top_10_movies = list(similarity_scores.iloc[1:11].index)
    recommended_movies = [list(final_df.index)[i] for i in top_10_movies]
    return recommended_movies
  except:
    print("No Movie Name Found")

In [ ]:
recommendation("Fast & Furious Hello")

No Movie Name Found


In [ ]:
recommendation("Fast & Furious")

['If I Stay',
 'Fast Five',
 'The Princess Diaries',
 'The Princess Diaries 2: Royal Engagement',
 'Texas Killing Fields',
 'The Informant!',
 'Thunderstruck',
 'Getaway',
 ' The Fast and the Furious',
 'Love & Mercy']

In [ ]:
recommendation("Harry Potter and the chamber of Secrets")

No Movie Name Found


In [ ]:
recommendation("Spider-Man")


['Spider-Man 2',
 'The Amazing Spider-Man 2',
 'Envy',
 'Elf-Man',
 'Spider-Man 3',
 ' The Tailor of Panama',
 'The Road',
 'Amazing Spider-Man, TheThe Amazing Spider-Man',
 'The Alibi',
 "Harry Potter and the Sorcerer's Stone"]

In [ ]:
recommendation("Ice Age")


['Ice Age: Continental Drift',
 'Ice Age: Dawn of the Dinosaurs',
 'Lords of Dogtown',
 'Ice Age: The Meltdown',
 'Brown Sugar',
 'Bella',
 'Diary of a Wimpy Kid: The Long Haul',
 ' I Can Do Bad All By Myself',
 'Shark Bait',
 'Scoop']